# Draft analysis 

---

Group name: Lena Breitberg, Ji Huixiao, Paraskevas Papadopoulos

---


## Introduction

*This section includes an introduction to the project motivation, data, and research question. Include a data dictionary* 

## Setup

In [20]:
import numpy as np
import pandas as pd

from scipy import stats
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns  

import altair as alt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Data

## Import data

In [78]:
# import data
ROOT = "https://raw.githubusercontent.com/hdm-statistik-lpj/project/main/data/external/"
DATA = "immobilienNuernberg.csv"
df = pd.read_csv(ROOT + DATA)

### Data structure

In [79]:
# show the first rows
pd.set_option('display.max_columns', None)
df.head()

,General.EstateTypeKey,General.ReferenceNumber,General.DistributionTypeKey,General.ConstructionYear,General.LivingSpace,Tealium.object_locationid,Tealium.object_address_is_visible,Tealium.object_zip,Tealium.object_currency,Tealium.object_features,Tealium.object_objektart_sub,Tealium.expose_type,Tealium.broker_guid,Tealium.object_count_photos,LocalRatings.scores.local_amenities,LocalRatings.scores.mobility,MetaBadges,Price.AdditionalInformation.Commission.CommissionType,EstateAddress.City,EstateAddress.ZipCode,EstateAddress.Street,Offerer.addressGuid,Offerer.globalUserId,Offerer.contactData.companyName,Project.Address.PostCode,Project.Address.Street,Project.Address.HouseNumber,Latitude,Longitude,Stadtteil,Stadt,PRICE,ROOMS,PRICE_COMMONCHARGE,EnergyType,Class,Value,CATEGORY,FLOOR,USAGE,SUITABILITY,CONDITION,HOUSECONDITION,PARKINGSLOT,ENERGY,BUILDINGTYPE,HEATING,ELEVATOR,EQUIPMENTS
0,WOHNUNG,001/G1/E114,ZUM_KAUF,2023.0,247.07,493480,True,90482,EUR,"['Neubau', 'WANNE', 'gaestewc', 'Bad/WC getren...",wohnung,project,4821481,15.0,7.7,9.5,"[{'label': 'Provisionsfrei', 'highlight': True}]",COMMISSION_FREE,Nürnberg,90482,Ostendstraße 120,69C9B07CBE7147ED868C55D3368C8261,4821481,Instone Real Estate Development GmbH Niederlas...,90482.0,Ostendstraße,115,49.45760,11.13188,Mögeldorf,Nürnberg,2539000.0,7.0,NaN,NaN,NaN,NaN,NaN,11. Geschoss,NaN,NaN,Erstbezug,Neubau,Tiefgarage,NaN,NaN,Fußbodenheizung,Personenaufzug,NaN
1,WOHNUNG,NaN,ZUM_KAUF,1900.0,69.00,493430,False,90461,EUR,"['Dach ausgebaut', 'WANNE', 'gaestewc', 'Bad/W...",MAISONETTE,standard,2704438,23.0,6.5,9.7,[],COMMISSION_REQUIRED,Nürnberg,90461,NaN,96453B9B5899480AB506126D5E0E9140,2704438,Immobilien Büro Franken24 UG (haftungsbeschränkt),NaN,NaN,NaN,49.42749,11.07581,Gibitzenhof,Nürnberg,225000.0,2.0,280.0,NaN,NaN,NaN,Maisonette,4. Geschoss (Dachgeschoss),vermietet,WG-geeignet,renoviert / saniert,NaN,NaN,Gas,NaN,Etagenheizung,NaN,NaN
2,WOHNUNG,NaN,ZUM_KAUF,1972.0,69.46,493465,False,90473,EUR,"['WANNE', 'GEPFLEGT', 'FERN', 'LAMINAT', 'FLIE...",wohnung,standard,7111707,1.0,5.7,7.8,[],COMMISSION_DEFAULT,Nürnberg,90473,NaN,9C587ED116BD427E9DD13790553F0A5F,7111707,W &a; F Liegenschaften GmbH,NaN,NaN,NaN,49.40815,11.13376,Langwasser,Nürnberg,195000.0,3.0,283.0,NaN,NaN,NaN,NaN,8. Geschoss,vermietet,NaN,gepflegt,NaN,Tiefgarage,Fernwärme,NaN,NaN,NaN,NaN
3,WOHNUNG,X4_301,ZUM_KAUF,2023.0,82.81,493524,True,90429,EUR,"['Neubau', 'DUSCHE', 'Personenaufzug', 'ERSTBE...",wohnung,project,53124,6.0,8.2,9.6,"[{'label': 'Provisionsfrei', 'highlight': True}]",COMMISSION_FREE,Nürnberg,90429,Maximilianstraße 56,F2237DED191344059DB32E4F5C9709F0,53124,EcoLoft Gruppe AG,90429.0,Maximilianstraße,56,49.46008,11.04266,Seeleinsbühl,Nürnberg,483623.0,3.0,NaN,UNDEFINED,DE_ENV2014_CLASS_A_PLUS,24.5,NaN,3. Geschoss,NaN,NaN,Erstbezug,Neubau,NaN,Blockheizkraftwerk,KfW 55,Fußbodenheizung,Personenaufzug,NaN
4,WOHNUNG,X4_204,ZUM_KAUF,2023.0,75.52,493524,True,90429,EUR,"['Neubau', 'DUSCHE', 'Personenaufzug', 'Erdges...",wohnung,project,53124,6.0,8.2,9.6,"[{'label': 'Provisionsfrei', 'highlight': True}]",COMMISSION_FREE,Nürnberg,90429,Maximilianstraße 56,F2237DED191344059DB32E4F5C9709F0,53124,EcoLoft Gruppe AG,90429.0,Maximilianstraße,56,49.46008,11.04266,Seeleinsbühl,Nürnberg,433956.0,3.0,NaN,UNDEFINED,DE_ENV2014_CLASS_A_PLUS,24.5,NaN,2. Geschoss (Erdgeschoss),NaN,NaN,Erstbezug,Neubau,NaN,Blockheizkraftwerk,KfW 55,Fußbodenheizung,Personenaufzug,NaN


In [80]:
# data overview (with meta data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 961 entries, 0 to 960
Data columns (total 49 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   General.EstateTypeKey                                  961 non-null    object 
 1   General.ReferenceNumber                                832 non-null    object 
 2   General.DistributionTypeKey                            961 non-null    object 
 3   General.ConstructionYear                               961 non-null    float64
 4   General.LivingSpace                                    961 non-null    float64
 5   Tealium.object_locationid                              961 non-null    int64  
 6   Tealium.object_address_is_visible                      961 non-null    bool   
 7   Tealium.object_zip                                     961 non-null    int64  
 8   Tealium.object_currency                           

In [81]:
# show all variables in the data set
df.columns

Index(['General.EstateTypeKey', 'General.ReferenceNumber',
       'General.DistributionTypeKey', 'General.ConstructionYear',
       'General.LivingSpace', 'Tealium.object_locationid',
       'Tealium.object_address_is_visible', 'Tealium.object_zip',
       'Tealium.object_currency', 'Tealium.object_features',
       'Tealium.object_objektart_sub', 'Tealium.expose_type',
       'Tealium.broker_guid', 'Tealium.object_count_photos',
       'LocalRatings.scores.local_amenities', 'LocalRatings.scores.mobility',
       'MetaBadges', 'Price.AdditionalInformation.Commission.CommissionType',
       'EstateAddress.City', 'EstateAddress.ZipCode', 'EstateAddress.Street',
       'Offerer.addressGuid', 'Offerer.globalUserId',
       'Offerer.contactData.companyName', 'Project.Address.PostCode',
       'Project.Address.Street', 'Project.Address.HouseNumber', 'Latitude',
       'Longitude', 'Stadtteil', 'Stadt', 'PRICE', 'ROOMS',
       'PRICE_COMMONCHARGE', 'EnergyType', 'Class', 'Value', 'CATEGORY',

### Data corrections

In [82]:
#rename variables with categorization
df.rename(columns={
    # General Property Information
    'General.EstateTypeKey': 'estateTypeKey',
    'General.ReferenceNumber': 'referenceNumber',
    'General.DistributionTypeKey': 'distributionTypeKey',
    'General.ConstructionYear': 'constructionYear',
    'General.LivingSpace': 'livingSpace',

    # Tealium Object Information
    'Tealium.object_locationid': 'objectLocationId',
    'Tealium.object_address_is_visible': 'objectAddressIsVisible',
    'Tealium.object_zip': 'objectZip',
    'Tealium.object_currency': 'objectCurrency',
    'Tealium.object_features': 'objectFeatures',
    'Tealium.object_objektart_sub': 'objectObjektartSub',
    'Tealium.expose_type': 'exposeType',
    'Tealium.broker_guid': 'brokerGuid',
    'Tealium.object_count_photos': 'photosCount',

    # Local Ratings
    'LocalRatings.scores.local_amenities': 'localAmenitiesScore',
    'LocalRatings.scores.mobility': 'localMobilityScore',

    # Meta Information
    'MetaBadges': 'metaBadges',

    # Price and Commission Information
    'Price.AdditionalInformation.Commission.CommissionType': 'commissionType',
    'PRICE': 'price',
    'PRICE_COMMONCHARGE': 'priceCommoncharge',

    # Estate Address Information
    'EstateAddress.City': 'estateCity',
    'EstateAddress.ZipCode': 'estateZipCode',
    'EstateAddress.Street': 'estateStreet',

    # Offerer Information
    'Offerer.addressGuid': 'offererAddressGuid',
    'Offerer.globalUserId': 'offererGlobalUserId',
    'Offerer.contactData.companyName': 'offererCompanyName',

    # Project Address Information
    'Project.Address.PostCode': 'projectPostCode',
    'Project.Address.Street': 'projectStreet',
    'Project.Address.HouseNumber': 'projectHouseNumber',

    # Location Information
    'Latitude': 'latitudeDistrict',
    'Longitude': 'longitudeDistrict',
    'Stadtteil': 'district',
    'Stadt': 'town',

    # Energy Information
    'EnergyType': 'energyType',
    'Class': 'energyClass',
    'Value': 'energyConsumption',

    # Room Information
    'ROOMS': 'rooms',

    # Other Property Details
    'CATEGORY': 'category',
    'FLOOR': 'floor',
    'USAGE': 'usage',
    'SUITABILITY': 'suitability',
    'CONDITION': 'condition',
    'HOUSECONDITION': 'houseCondition',
    'PARKINGSLOT': 'parkingSlot',
    'ENERGY': 'energy',
    'BUILDINGTYPE': 'buildingType',
    'HEATING': 'heating',
    'ELEVATOR': 'elevator',
    'EQUIPMENTS': 'equipments'},
    inplace=True
    )

#### Duplikate

Wir suchen nach Duplikaten in den Referenznummern (referenceNumber) innerhalb desselben Maklers (Gruppierungsvariable: brokerGuid), da Duplikate nur innerhalb desselben Maklers auftreten können. 
Annahme: Eigentümer beauftragen nur einen Makler mit dem Verkauf, um mögliche höhere Maklerkosten zu vermeiden.

Zudem wird überprüft ob der Datensatz identische Datensätze enthält.

Variablen, die für diese Prüfungen verwendet wurden, werden im Anschluss aus dem DataFrame entfernt, wenn diese voraussichtlich keine weiteren Mehrwert bitten.

In [83]:
duplicateCount = df.groupby(['brokerGuid', 'referenceNumber']).size().reset_index(name='Duplicate Count').sort_values(by='Duplicate Count', ascending=False)
#print(duplicateCount)
if duplicateCount['Duplicate Count'].eq(1).all():
    print("Keine Duplikate innerhalb desselben Maklers gefunden")

# Überprüfung auf identische Zeilen
identicalRows = df[df.duplicated()]
if identicalRows.empty:
    print("Keine identischen Zeilen gefunden")

Keine Duplikate innerhalb desselben Maklers gefunden
Keine identischen Zeilen gefunden


In [84]:
df.drop(columns=['referenceNumber', 'brokerGuid'], inplace=True)

Für die fortlaufende Analyse sind wir an folgenden Kriterien interessiert:

- Nur Wohnungen werden berücksichtigt; andere Arten von Immobilien werden aus der Analyse ausgeschlossen.
- Wohnungen, die zwangsversteigert werden, werden nicht einbezogen, da ihre Verkaufspreise nicht marktnah sind.
- Es werden ausschließlich Wohnungen betrachtet, die zum Verkauf stehen.
- Beschränkung auf Wohnungen in der Stadt Nürnberg.

Immobilien, die diese Kriterien nicht erfüllen, werden aus dem Datensatz entfernt. Variablen, die für diese Prüfungen verwendet wurden, werden im Anschluss aus dem DataFrame entfernt, wenn diese voraussichtlich keine weiteren Mehrwert bitten.

In [85]:
# Count values
valueCountsList = [
    df['estateTypeKey'].value_counts().rename('estateTypeCount'),
    df['metaBadges'].value_counts().rename('metaBadgesCount'),
    df['distributionTypeKey'].value_counts().rename('distributionTypeKeyCount'),
    df['town'].value_counts().rename('townCount')
]

# Concatenate the value counts into one DataFrame with a multi-level index
combinedCounts = pd.concat(valueCountsList, axis=1, keys=[s.name for s in valueCountsList])

print(combinedCounts)

                                                    estateTypeCount  \
WOHNUNG                                                       959.0   
HAUS                                                            2.0   
[]                                                              NaN   
[{'label': 'Provisionsfrei', 'highlight': True}]                NaN   
[{'label': 'Privater Anbieter', 'highlight': Tr...              NaN   
[{'label': 'Zwangsversteigerung', 'highlight': ...              NaN   
ZUM_KAUF                                                        NaN   
Nürnberg                                                        NaN   

                                                    metaBadgesCount  \
WOHNUNG                                                         NaN   
HAUS                                                            NaN   
[]                                                            667.0   
[{'label': 'Provisionsfrei', 'highlight': True}]              253.0   
[{'la

In [86]:
# Löschen der Zeilen mit "estateTypeKey" "HAUS"
df.drop(df[df['estateTypeKey'].str.contains('HAUS')].index, inplace=True)
# Löschen Variablen
df = df.drop(columns=['estateTypeKey'])

In [87]:
# Löschen der Zeilen mit Label "Zwangsversteigerung" in der Spalte MetaBadges
df = df[~df['metaBadges'].astype(str).str.contains("'label': 'Zwangsversteigerung'")]
# Löschen Variablen
df.drop(columns=['metaBadges'], inplace=True)

In [88]:
# Löschen Variablen
df.drop(columns=['distributionTypeKey'], inplace=True)

#### Geographische Variablen
Geographische Attribute wie Längengrad (latitudeDistrict) und Breitengrad (longitudeDistrict) des Stadtteils werden entfernt, da diese Informationen bereits in der Variable "Stadtteil" (district) enthalten sind. Die Distanz zwischen Stadtteilen wird nicht berücksichtigt. Da alle Immobilien in Nürnberg sind, wird die Variable "Stadt" (town) gelöscht. Da nur der Stadtteil von Interesse ist, werden folgende geographische Variablen entfernt:

- 'projectPostCode'
- 'projectStreet'
- 'projectHouseNumber'
- 'estateCity'
- 'estateZipCode'
- 'estateStreet'
- 'latitudeDistrict'
- 'longitudeDistrict'
- 'town'
- 'objectAddressIsVisible'
- 'objectAddressIsVisible'
- 'objectZip'

Die Löschung dieser geographischen Variablen dient der Datenbereinigung und ermöglicht eine effiziente und fokussierte Analyse auf die relevanten Informationen. 

In [89]:
columnsToDrop = [
    'projectPostCode', 'projectStreet', 'projectHouseNumber',
    'estateCity', 'estateZipCode', 'estateStreet',
    'latitudeDistrict', 'longitudeDistrict', 'town',
    'objectAddressIsVisible', 'objectZip', 'objectLocationId'
]

# Drop specified columns from the DataFrame
df.drop(columns=columnsToDrop, axis=1, inplace=True)

In [90]:
df

,constructionYear,livingSpace,objectCurrency,objectFeatures,objectObjektartSub,exposeType,photosCount,localAmenitiesScore,localMobilityScore,commissionType,offererAddressGuid,offererGlobalUserId,offererCompanyName,district,price,rooms,priceCommoncharge,energyType,energyClass,energyConsumption,category,floor,usage,suitability,condition,houseCondition,parkingSlot,energy,buildingType,heating,elevator,equipments
0,2023.0,247.07,EUR,"['Neubau', 'WANNE', 'gaestewc', 'Bad/WC getren...",wohnung,project,15.0,7.7,9.5,COMMISSION_FREE,69C9B07CBE7147ED868C55D3368C8261,4821481,Instone Real Estate Development GmbH Niederlas...,Mögeldorf,2539000.0,7.0,NaN,NaN,NaN,NaN,NaN,11. Geschoss,NaN,NaN,Erstbezug,Neubau,Tiefgarage,NaN,NaN,Fußbodenheizung,Personenaufzug,NaN
1,1900.0,69.00,EUR,"['Dach ausgebaut', 'WANNE', 'gaestewc', 'Bad/W...",MAISONETTE,standard,23.0,6.5,9.7,COMMISSION_REQUIRED,96453B9B5899480AB506126D5E0E9140,2704438,Immobilien Büro Franken24 UG (haftungsbeschränkt),Gibitzenhof,225000.0,2.0,280.0,NaN,NaN,NaN,Maisonette,4. Geschoss (Dachgeschoss),vermietet,WG-geeignet,renoviert / saniert,NaN,NaN,Gas,NaN,Etagenheizung,NaN,NaN
2,1972.0,69.46,EUR,"['WANNE', 'GEPFLEGT', 'FERN', 'LAMINAT', 'FLIE...",wohnung,standard,1.0,5.7,7.8,COMMISSION_DEFAULT,9C587ED116BD427E9DD13790553F0A5F,7111707,W &a; F Liegenschaften GmbH,Langwasser,195000.0,3.0,283.0,NaN,NaN,NaN,NaN,8. Geschoss,vermietet,NaN,gepflegt,NaN,Tiefgarage,Fernwärme,NaN,NaN,NaN,NaN
3,2023.0,82.81,EUR,"['Neubau', 'DUSCHE', 'Personenaufzug', 'ERSTBE...",wohnung,project,6.0,8.2,9.6,COMMISSION_FREE,F2237DED191344059DB32E4F5C9709F0,53124,EcoLoft Gruppe AG,Seeleinsbühl,483623.0,3.0,NaN,UNDEFINED,DE_ENV2014_CLASS_A_PLUS,24.5,NaN,3. Geschoss,NaN,NaN,Erstbezug,Neubau,NaN,Blockheizkraftwerk,KfW 55,Fußbodenheizung,Personenaufzug,NaN
4,2023.0,75.52,EUR,"['Neubau', 'DUSCHE', 'Personenaufzug', 'Erdges...",wohnung,project,6.0,8.2,9.6,COMMISSION_FREE,F2237DED191344059DB32E4F5C9709F0,53124,EcoLoft Gruppe AG,Seeleinsbühl,433956.0,3.0,NaN,UNDEFINED,DE_ENV2014_CLASS_A_PLUS,24.5,NaN,2. Geschoss (Erdgeschoss),NaN,NaN,Erstbezug,Neubau,NaN,Blockheizkraftwerk,KfW 55,Fußbodenheizung,Personenaufzug,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,2023.0,110.32,EUR,"['Neubau', 'WANNE', 'gaestewc', 'DUSCHE', 'Kel...",wohnung,project,12.0,7.7,9.5,COMMISSION_FREE,69C9B07CBE7147ED868C55D3368C8261,4821481,Instone Real Estate Development GmbH Niederlas...,Mögeldorf,909000.0,4.0,NaN,NaN,NaN,NaN,NaN,12. Geschoss,NaN,NaN,Erstbezug,Neubau,Tiefgarage,NaN,NaN,Fußbodenheizung,Personenaufzug,NaN
957,2023.0,103.11,EUR,"['Neubau', 'barriefrei', 'WANNE', 'gaestewc', ...",wohnung,project,8.0,7.7,9.5,COMMISSION_FREE,69C9B07CBE7147ED868C55D3368C8261,4821481,Instone Real Estate Development GmbH Niederlas...,Mögeldorf,819000.0,4.0,NaN,NaN,NaN,NaN,NaN,7. Geschoss,NaN,barrierefrei,Erstbezug,Neubau,Tiefgarage,NaN,NaN,Fußbodenheizung,Personenaufzug,NaN
958,1965.0,91.00,EUR,"['Personenaufzug', 'Zentralheizung', 'LINOLEUM...",wohnung,standard,8.0,4.3,9.3,COMMISSION_REQUIRED,2D9F0B6156E34C40A9D0410518CFFA30,325129,Wolfgang Evirgen Immobilien,Gibitzenhof,295000.0,4.0,NaN,UNDEFINED,NaN,91.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zentralheizung,Personenaufzug,NaN
959,1958.0,34.00,EUR,"['DUSCHE', 'Kelleranteil', 'GAS', 'Zentralheiz...",Apartment,standard,18.0,8.9,9.7,COMMISSION_FREE,311C85D6B95E4AF894B62F1CA747AEA5,5309078,FBK Development GmbH,Maxfeld,198000.0,1.0,137.0,UNDEFINED,DE_ENV2014_CLASS_D,118.3,Apartment,4. Geschoss,frei,NaN,NaN,NaN,NaN,Gas,NaN,Zentralheizung,NaN,möbliert


In [11]:
# Warum änderst du die Skallierung der Variablen?
df_new = pd.DataFrame(data={
    "contructionYear": df["General.ConstructionYear"],
    "livingSpace": df["General.LivingSpace"],
    "photosCount": df["Tealium.object_count_photos"].astype("Int64"),
    "localAmenetiesScore": df["LocalRatings.scores.local_amenities"] / 10,
    "localMobilityScore": df["LocalRatings.scores.mobility"] / 10,
    "latitude": df["Latitude"],
    "longitude": df["Longitude"],
    "rooms": df["ROOMS"],
    "priceCommoncharge": df["PRICE_COMMONCHARGE"],
    "energyConsumption": df["Value"],
    "elevator": df["ELEVATOR"],
    "equipments": df["EQUIPMENTS"],
    "price": df["PRICE"],
    "energyClass": df["Class"],
})

In [ ]:
# change data type
df['...'] = pd.Categorical(df['...'])
df['...'] = pd.Categorical(df['...'])
df['...'] = pd.Categorical(df['...'])

In [91]:
# show missing values
print(df.isnull().sum())

constructionYear         0
livingSpace              0
objectCurrency           0
objectFeatures           0
objectObjektartSub       0
exposeType               0
photosCount              1
localAmenitiesScore      3
localMobilityScore       3
commissionType           0
offererAddressGuid       0
offererGlobalUserId      0
offererCompanyName      31
district                 0
price                    0
rooms                    0
priceCommoncharge      525
energyType             221
energyClass            326
energyConsumption      221
category               798
floor                  145
usage                  623
suitability            785
condition              533
houseCondition         731
parkingSlot            544
energy                 161
buildingType           830
heating                214
elevator               484
equipments             864
dtype: int64


In [84]:
equipments = {
    "möbliert": 1,
    "teilweise möbliert": 2, 
    "neuwertig": 3,
    "teilweise möbliert, neuwertig": 4,
    "möbliert, neuwertig": 5
}

In [85]:
energyClasses = {
    "A_PLUS": 1,
    "A": 2,
    "B": 3,
    "C": 4,
    "D": 5,
    "E": 6,
    "F": 7,
    "G": 8,
    "H": 9
}
df_correct["energyClass"] = df_correct["energyClass"].str \
    .removeprefix("DE_ENV2014_CLASS_") \
    .replace(energyClasses)


In [86]:
df_correct["elevator"] = df_correct["elevator"].notnull().astype(int)

In [87]:
df_correct

,contructionYear,livingSpace,photosCount,localAmenetiesScore,localMobilityScore,latitude,longitude,rooms,priceCommoncharge,energyConsumption,elevator,equipments,price,energyClass
0,2023.0,247.07,15,0.77,0.95,49.45760,11.13188,7.0,NaN,NaN,1,NaN,2539000.0,NaN
1,1900.0,69.00,23,0.65,0.97,49.42749,11.07581,2.0,280.0,NaN,0,NaN,225000.0,NaN
2,1972.0,69.46,1,0.57,0.78,49.40815,11.13376,3.0,283.0,NaN,0,NaN,195000.0,NaN
3,2023.0,82.81,6,0.82,0.96,49.46008,11.04266,3.0,NaN,24.5,1,NaN,483623.0,1.0
4,2023.0,75.52,6,0.82,0.96,49.46008,11.04266,3.0,NaN,24.5,1,NaN,433956.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,2023.0,110.32,12,0.77,0.95,49.45760,11.13188,4.0,NaN,NaN,1,NaN,909000.0,NaN
957,2023.0,103.11,8,0.77,0.95,49.45760,11.13188,4.0,NaN,NaN,1,NaN,819000.0,NaN
958,1965.0,91.00,8,0.43,0.93,49.42749,11.07581,4.0,NaN,91.3,1,NaN,295000.0,NaN
959,1958.0,34.00,18,0.89,0.97,49.46468,11.08784,1.0,137.0,118.3,0,möbliert,198000.0,5.0


### Variable lists

In [88]:
# remove outlier for visualisation
df_correct_vis = df_correct.loc[df_correct["price"] < 80000000]

In [89]:
y_labels = ["price", "energyClass"]
x_labels = ["constructionYear", "livingSpace",
            "photosCount", "localAmenitiesScore",
            "localMobilityScore", "latitude",
            "longitude", "priceCommoncharge",
            "energyConsumption", "elevator",
            "equipments"]

### Data splitting

## Analysis

### Descriptive statistics

In [90]:
df_correct.describe().T

,count,mean,std,min,25%,50%,75%,max
contructionYear,961.0,1979.03538,36.413922,1872.0,1957.0,1976.0,2019.0,2025.0
livingSpace,961.0,77.198096,35.495184,19.35,56.58,72.51,90.0,532.0
photosCount,960.0,12.927083,9.509278,1.0,7.0,11.0,16.0,54.0
localAmenetiesScore,958.0,0.672192,0.166368,0.1,0.57,0.7,0.8,0.95
localMobilityScore,958.0,0.917996,0.09166,0.15,0.91,0.96,0.97,0.97
latitude,961.0,49.443274,0.024613,49.34839,49.42828,49.44504,49.46145,49.52838
longitude,961.0,11.082636,0.036428,11.00723,11.05372,11.08505,11.10224,11.26159
rooms,961.0,2.761186,1.215971,1.0,2.0,3.0,3.0,22.0
priceCommoncharge,423.0,259.189835,111.251378,65.0,179.5,246.0,320.0,1078.0
energyConsumption,727.0,102.342352,54.312412,0.0,67.6,103.3,135.25,326.0


### Exploratory data analysis

In [8]:
alt.renderers.enable('html')
alt.Chart(df_correct_vis).mark_circle().encode(
    x=alt.X(alt.repeat("column"), 
            type='quantitative',
            scale=alt.Scale(zero=False)
            ),
    y=alt.Y(alt.repeat("row"), 
            type='quantitative',
            scale=alt.Scale(zero=False)
             )
).properties(
    width=150,
    height=150
).repeat(
    row=y_labels,
    column=x_labels
)

NameError: name 'df_correct_vis' is not defined

### Relationships

## Model

### Select model

### Training and validation

### Fit model

### Evaluation on test set

### Save model



Save your model in the folder `models/`. Use a meaningful name and a timestamp.

## Conclusions